# Collecting Images For Subtraction

This notebooks demonstrates how to collect template images and science images from [Rob's database](https://github.com/Roman-Supernova-PIT/roman-desc-simdex/blob/main/documentation.ipynb) for image subtraction. A detailed approach to searh Roman simulated images can be found [here](https://github.com/Roman-Supernova-PIT/roman-desc-simdex/blob/main/documentation.ipynb).

In [126]:
from collections import defaultdict

import requests
import pandas as pd

import numpy as np

from astropy.io import fits
from astropy.table import Table
from astropy.wcs import WCS

from astropy.coordinates import SkyCoord
from astropy.wcs.utils import skycoord_to_pixel

def get_sne(z_cmb_min=0.15, z_cmb_max=0.16):
    server_url = 'https://roman-desc-simdex.lbl.gov'
    req = requests.Session()
    result = req.post( f'{server_url}/findtransients/z_cmb_min={z_cmb_min}/z_cmb_max={z_cmb_max}/gentype=10',
                         json={ 'fields': ('id', 'ra', 'dec', 'z_cmb',  ) } ) # 'host_sn_sep' 'peak_mjd', 'peak_mag_g', 'model_name'
    if result.status_code != 200:
        raise RuntimeError( f"Got status code {result.status_code}\n{result.text}" )
    df = pd.DataFrame( result.json() )
    return df

def get_image_info_for_ra_dec(ra, dec, band=None):
    server_url = 'https://roman-desc-simdex.lbl.gov'
    req = requests.Session()
    json = { 'containing': [ ra, dec ] }
    if band is not None:
        json['filter'] = band  # 'filter' is reserved in Python, so we use 'band' for var name
    result = req.post( f'{server_url}/findromanimages', json=json )
    if result.status_code != 200:
        raise RuntimeError( f"Got status code {result.status_code}\n{result.text}" )
    df = pd.DataFrame( result.json() )
    return df

# This function doesn't work because we don't get access to BaseView.
def get_image_info_for_pointing(pointing):
    server_url = 'https://roman-desc-simdex.lbl.gov'
    req = requests.Session()
    json = { 'pointing': pointing }
    result = req.post( f'{server_url}/baseview', json=json )
    if result.status_code != 200:
        raise RuntimeError( f"Got status code {result.status_code}\n{result.text}" )
    df = pd.DataFrame( result.json() )
    return df

def load_wcs(image_path, hdu_id=0):
    with fits.open(image_path) as hdul:
        header = hdul[hdu_id].header
        wcs = WCS(header)
    return wcs

def load_table(table_path):
    table = Table.read(table_path, format='ascii').to_pandas()
    return table

def radec_to_xy(ra, dec, wcs, origin=0):
    # ra and dec are in degree unit
    sky_coord = SkyCoord(ra, dec, frame='icrs', unit='deg')
    pixel_coords = skycoord_to_pixel(coords=sky_coord, wcs=wcs, origin=origin)
    return pixel_coords[0], pixel_coords[1]

def xy_in_image(x, y, width, height, offset=0):
    return (0 + offset <= x) & (x < width - offset) & (0 + offset <= y) & (y < height - offset)

## Get information of SNe

In [127]:
sne = get_sne(z_cmb_min=0.10, z_cmb_max=0.17)
print(f'Found {len(sne)} SNe.')

Found 277 SNe.


## Pick 1 SN

In [128]:
band = 'R062'

for idx in range(len(sne)):
    sn = sne.iloc[idx]
    image_info = get_image_info_for_ra_dec(sn.ra, sn.dec, band=band)
    image_info = image_info.sort_values(by=['mjd'])
    # We need at least 11 images which cover the same sn.
    # We use the first image as template, and the remaining as science images.
    if len(image_info) < 11:
        continue
    break


In [129]:
image_info

,boredec,borera,dec,dec_00,dec_01,dec_10,dec_11,exptime,filter,mjd,pa,pointing,ra,ra_00,ra_01,ra_10,ra_11,sca
0,-43.1930,7.98151,-43.172957,-43.232820,-43.111897,-43.233925,-43.113007,161.025,R062,62000.04546,0.00000,8,8.446753,8.364626,8.360879,8.532839,8.528759,8
1,-43.1930,8.35779,-43.228562,-43.289640,-43.167200,-43.289833,-43.167393,161.025,R062,62000.06150,0.00000,13,8.451116,8.365784,8.365217,8.537230,8.536320,1
2,-43.4006,8.23644,-43.173991,-43.221107,-43.105743,-43.242220,-43.126820,161.025,R062,62005.06150,9.86301,398,8.384804,8.288700,8.314817,8.454900,8.480711,3
3,-43.3362,8.60716,-43.218282,-43.267124,-43.149100,-43.287400,-43.169330,161.025,R062,62005.07754,9.86301,403,8.351652,8.252760,8.283496,8.420011,8.450435,14
4,-43.4989,8.50665,-43.228093,-43.262960,-43.153170,-43.303010,-43.193146,161.025,R062,62010.07750,19.72602,788,8.343160,8.235339,8.292100,8.394293,8.450805,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,-43.1930,8.35779,-43.228562,-43.289640,-43.167200,-43.289833,-43.167393,161.025,R062,63550.06150,0.00000,56223,8.451116,8.365784,8.365217,8.537230,8.536320,1
164,-43.4006,8.23644,-43.173991,-43.221107,-43.105743,-43.242220,-43.126820,161.025,R062,63555.06150,9.86301,56608,8.384804,8.288700,8.314817,8.454900,8.480711,3
165,-43.3362,8.60716,-43.218282,-43.267124,-43.149100,-43.287400,-43.169330,161.025,R062,63555.07754,9.86301,56613,8.351652,8.252760,8.283496,8.420011,8.450435,14
166,-43.4989,8.50665,-43.228093,-43.262960,-43.153170,-43.303010,-43.193146,161.025,R062,63560.07750,19.72602,56998,8.343160,8.235339,8.292100,8.394293,8.450805,15


We pick the first image as the template. For the science image, we select from the remaining images in reverse chronological order toward the template image. We require at least 50% of the truth in the template should exist in the science. This ensure both images cover enough overlapping region.

In [130]:
INPUT_IMAGE_PATTERN = ("/global/cfs/cdirs/lsst/shared/external/roman-desc-sims/Roman_data"
                                "/RomanTDS/images/simple_model/{band}/{pointing}/Roman_TDS_simple_model_{band}_{pointing}_{sca}.fits.gz")
INPUT_TRUTH_PATTERN = ("/global/cfs/cdirs/lsst/shared/external/roman-desc-sims/Roman_data"
                             "/RomanTDS/truth/{band}/{pointing}/Roman_TDS_index_{band}_{pointing}_{sca}.txt")

MATCH_RADIUS = 0.4
image_width =4088
image_height = 4088

selected_rows = []
template_info = image_info.iloc[0]

for i in range(len(image_info)-1, 0, -1):
    science_info = image_info.iloc[i]
    science_id = {'band': band, 'pointing': int(science_info['pointing']), 'sca': int(science_info['sca'])}
    template_id = {'band': band, 'pointing': int(template_info['pointing']), 'sca': int(template_info['sca'])}

    science_truth_path = INPUT_TRUTH_PATTERN.format(**science_id)                
    science_truth = load_table(science_truth_path)

    template_image_path = INPUT_IMAGE_PATTERN.format(**template_id)
    template_wcs = load_wcs(template_image_path, hdu_id=1)

    # Some sources from the truth table are not in the image. We need to remove them.
    science_in_science = xy_in_image(science_truth.x, science_truth.y, width=image_width, height=image_height, offset=0)
    science_truth = science_truth[science_in_science].copy().reset_index(drop=True)

    x_in_template, y_in_template = radec_to_xy(science_truth.ra, science_truth.dec, template_wcs, origin=1)
    science_in_template = xy_in_image(x_in_template, y_in_template, width=image_width, height=image_height, offset=0)
    
    if science_in_template.sum() / len(science_in_template) > 0.5:
        selected_rows.append(i)
    if len(selected_rows) == 10:
        break

FileNotFoundError: [Errno 2] No such file or directory: '/global/cfs/cdirs/lsst/shared/external/roman-desc-sims/Roman_data/RomanTDS/truth/R062/57003/Roman_TDS_index_R062_57003_17.txt'

In [ ]:
selected_rows.sort()
selected_image_info = image_info.iloc[selected_rows]

data_records = pd.DataFrame()
data_records['template_band'] = [band] * 10
data_records['template_pointing'] = [template_info.pointing] * 10
data_records['template_sca'] = [template_info.sca] * 10
# mjd is not required for running the detection pipeline
data_records['template_mjd'] = [template_info.mjd] * 10
data_records['science_band'] = [band] * 10
data_records['science_pointing'] = selected_image_info.pointing.values
data_records['science_sca'] = selected_image_info.sca.values
data_records['science_mjd'] = selected_image_info.mjd.values
data_records.to_csv('../test/test_ten_data_records.csv', index=False)

# For a given image, find all of the matching images.

In [134]:
def get_possible_template_images(im, min_points=3):
    """Return a list of matching images that could be used as templates.
    
    Returns all that overlap at least min_points of the 5 points of the center + corners of the images
    and that matches in bandpass
    
    images: Object with get method for
        ("ra", "dec", "ra_00", "dec_00", "ra_01", "dec_01", "ra_10", "dec_10", "ra_11", "dec_11", "filter")
    min_points: int
    """
    corners = [(im.ra_00, im.dec_00), (im.ra_01, im.dec_01), (im.ra_10, im.dec_10), (im.ra_11, im.dec_11)]
    center = [(im.ra, im.dec)]
    points = center + corners

    band = im.get("filter")  # 'filter' is method, so we can't use data attribute of same name to access

    matches = defaultdict(list)
    for i, (ra, dec) in enumerate(points):
        matching_images = get_image_info(ra, dec, band=band)
        for p, s, f in zip(matching_images.pointing, matching_images.sca, matching_images.get("filter")):
            matches[(p, s, f)].append(i)

    good_matches = []
    for k, v in matches.items():
        if len(v) >= min_points:
            good_matches.append(k)
            
    return good_matches

In [135]:
get_possible_template_images(im)

[(8, 8, 'R062'),
 (1174, 8, 'R062'),
 (2729, 6, 'R062'),
 (5437, 2, 'R062'),
 (7731, 8, 'R062'),
 (8500, 18, 'R062'),
 (8501, 7, 'R062'),
 (12332, 11, 'R062'),
 (12717, 3, 'R062'),
 (15423, 18, 'R062'),
 (15424, 7, 'R062'),
 (15809, 5, 'R062'),
 (20067, 13, 'R062'),
 (21221, 16, 'R062'),
 (22365, 17, 'R062'),
 (22745, 17, 'R062'),
 (25421, 1, 'R062'),
 (26577, 11, 'R062'),
 (29279, 8, 'R062'),
 (30834, 6, 'R062'),
 (33542, 2, 'R062'),
 (35836, 8, 'R062'),
 (36605, 18, 'R062'),
 (36606, 7, 'R062'),
 (40437, 11, 'R062'),
 (40822, 3, 'R062'),
 (43528, 18, 'R062'),
 (43529, 7, 'R062'),
 (43914, 5, 'R062'),
 (48172, 13, 'R062'),
 (49326, 16, 'R062'),
 (50470, 17, 'R062'),
 (50850, 17, 'R062'),
 (53526, 1, 'R062'),
 (54682, 11, 'R062'),
 (56218, 8, 'R062')]

In [104]:
# For one particular pointing, let's build up the match lists for all of the SCAs
pointing = 8
sca = range(18)
band = "R062"

In [125]:
# Doesn't work because BaseView isn't exposed
# get_image_info_for_pointing(pointing)

RuntimeError: Got status code 404
<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>
